In [ ]:
# ====================================================
# Recod.ai / LUC Scientific Image Forgery Detection
# Offline-Ready Notebook (PyTorch + Albumentations)
# ====================================================

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import models, transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.auto import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split

# ====================================================
# CONFIG
# ====================================================
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 8
EPOCHS = 12
LR = 1e-4
IMG_SIZE = 256
base_dir = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"

paths = {
    "train_authentic": os.path.join(base_dir, "train_images", "authentic"),
    "train_forged": os.path.join(base_dir, "train_images", "forged"),
    "train_masks": os.path.join(base_dir, "train_masks"),
    "test_images": os.path.join(base_dir, "test_images"),
    "sample_submission": os.path.join(base_dir, "sample_submission.csv")
}

In [ ]:
# ====================================================
# TRANSFORMS
# ====================================================
train_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.OneOf([
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
    ], p=0.3),
    A.ColorJitter(p=0.3),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [ ]:
# ====================================================
# DATASET
# ====================================================
class ForgeryDataset(Dataset):
    def __init__(self, authentic_dir, forged_dir, mask_dir, transform=None):
        self.authentic_paths = sorted([
            os.path.join(authentic_dir, f) for f in os.listdir(authentic_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ])
        self.forged_paths = sorted([
            os.path.join(forged_dir, f) for f in os.listdir(forged_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ])
        self.mask_dir = mask_dir
        self.transform = transform

        # Combine and assign labels
        self.image_paths = self.authentic_paths + self.forged_paths
        self.labels = [0]*len(self.authentic_paths) + [1]*len(self.forged_paths)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Mask exists only for forged images
        fname = os.path.basename(img_path)
        mask_path = os.path.join(self.mask_dir, fname)
        if os.path.exists(mask_path):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        else:
            mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)

        if self.transform:
            transformed = self.transform(image=img, mask=mask)
            img = transformed['image']
            mask = transformed['mask']

        return img, mask.unsqueeze(0).float(), torch.tensor(label, dtype=torch.long)

In [ ]:
# ====================================================
# DATA LOADERS
# ====================================================
dataset = ForgeryDataset(paths['train_authentic'], paths['train_forged'], paths['train_masks'], transform=train_transform)
indices = np.arange(len(dataset))
np.random.shuffle(indices)
train_size = int(0.8 * len(indices))
train_idx, val_idx = indices[:train_size], indices[train_size:]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=2)
val_loader = DataLoader(ForgeryDataset(paths['train_authentic'], paths['train_forged'], paths['train_masks'], transform=val_transform),
                        batch_size=BATCH_SIZE, sampler=val_sampler, num_workers=2)

print("Train samples:", len(train_idx), "| Val samples:", len(val_idx))

In [ ]:
# ====================================================
# MODEL (ResNet50 Binary Classifier - Offline)
# ====================================================
from torchvision.models import resnet50, ResNet50_Weights

# initialize ResNet50 backbone
model = resnet50(weights=None)  # don't download weights

# load local pretrained weights
local_weights_path = "/kaggle/input/resnet50-0676ba61-pth/resnet50-0676ba61.pth"
state_dict = torch.load(local_weights_path, map_location="cpu")

# remove possible 'fc' mismatch when loading partial weights
filtered_dict = {k: v for k, v in state_dict.items() if k in model.state_dict()}
model.load_state_dict(filtered_dict, strict=False)

# replace the final classification head
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 2)
)

model = model.to(DEVICE)

# ====================================================
# LOSS & OPTIMIZER
# ====================================================
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

# ====================================================
# TRAIN LOOP
# ====================================================
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    for imgs, masks, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * imgs.size(0)
    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, masks, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]"):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")

# ====================================================
# SAVE MODEL
# ====================================================
torch.save(model.state_dict(), "forgery_detector_resnet50.pth")
print("✅ Model saved to forgery_detector_resnet50.pth")

In [ ]:
# ====================================================
# INFERENCE & SUBMISSION
# ====================================================
test_dir = paths['test_images']
test_files = sorted(os.listdir(test_dir))

model.eval()
preds = []
for fname in tqdm(test_files, desc="Predicting"):
    img_path = os.path.join(test_dir, fname)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = val_transform(image=img)["image"].unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        output = model(img)
        pred = torch.softmax(output, dim=1)[:, 1].item()
    preds.append(pred)

df_sub = pd.read_csv(paths["sample_submission"])
df_sub["predicted"] = preds
df_sub.to_csv("submission.csv", index=False)
print("✅ submission.csv saved.")